In [1]:
from pyfuseki import FusekiQuery, FusekiUpdate
from pysolr import Solr
import httpx
from rdflib import Graph
from api.src.routes.translate.makeTranslate import MakeTranslate
from api.src.function.loc.graphExist import GraphExist
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from api.src.function.authorities.makeGraph import MakeGraphSubject

In [10]:
from api.src.function.authorities.makeGraph import MakeGraphSubject

In [ ]:
graph = MakeGraphSubject(request, id)
print(graph)

In [2]:
fuseki_query = FusekiQuery('http://localhost:3030', 'authorities')

In [3]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">306</int>\n</lst>\n</response>\n'

In [2]:
authority = 'http://id.loc.gov/authorities/subjects/sh85076723'

In [41]:
url = f'http://localhost:8000/authorities/loc/subject?uri={authority}'
r = httpx.get(url)
r.json()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-05-15',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-15T21:27:34',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh85111809'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Recatalogação', 'lang': 'pt'}}],
 'note': None,
 'hasBroaderAuthority': [{'value': 'https://bibliokeia.com/authorities/Topic/bkau-1',
   'label': {'value': 'Biblioteconomia', 'lang': 'por'},
   'base': 'bk'}],
 'hasBroaderExternalAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85020816',
   'label': {'value': 'Cataloging', 'lang': 'en'},
   'base': 'loc'}]

In [42]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=r.json())
response.status_code

201

In [15]:
graph = Graph()
graph.parse(f'{authority}.rdf')
graph.serialize('Recataloging.ttl')

<Graph identifier=N40e40ec0937c44b8b67f4545e8e564f7 (<class 'rdflib.graph.Graph'>)>

Recataloging

In [13]:
authority = 'http://id.loc.gov/authorities/subjects/sh85111809'

In [ ]:
url = f'http://localhost:8000/authorities/loc/subject?uri={authority}'
r = httpx.get(url)
r.json()

In [5]:
class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasReciprocalAuthority: Optional[list[Uri]]
    hasReciprocalExternalAuthority: Optional[list[Uri]]
    hasBroaderAuthority: Optional[list[Uri]]
    hasBroaderExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

def GetUriBK(url):
    token = url.split("/")[-1]
    queryBK = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
      PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
      SELECT ?uri ?label WHERE {{ graph ?g {{
          ?uri madsrdf:authoritativeLabel ?label .
        ?sub bf:identifiedBy ?identifiedBy .
          ?identifiedBy rdf:value "{token}"
          FILTER EXISTS {{ ?identifiedBy bf:assigner <http://id.loc.gov/vocabulary/organizations/dlc> }}
      }} }} """
    response = fuseki_query.run_sparql(queryBK)
    [binding] = response.convert()['results']['bindings']
    uri = {"value": binding['uri']['value'],
              "label": {
                "value": binding['label']['value'],
                "lang": "pt"
              } }
    return uri

def GetUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          url = i.get('value')
          if metadado in ['hasBroaderAuthority', 'hasNarrowerAuthority']:
            exist = GraphExist(url)
            if exist:
              uri = GetUriBK(url)
            else:
              uri = {
                    "value": url,
                    "label": {
                  "value": i.get('label').value,
                  "lang": i.get('label').language
                } }
          else:
            uri = {
                  "value": url,
                   "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj



In [43]:
request = Subject(**r.json())
request

Subject(type='Topic', adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/dlc', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 15), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 5, 15, 21, 27, 34), identifiedBy=[IdentifiedBy(type='Lccn', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='sh85111809')], status=Status(value='mstatus:new', label='novo')), elementList=[Element(type='TopicElement', elementValue=Label(value='Recatalogação', lang='pt'))], note=None, hasReciprocalAuthority=None, hasReciprocalExternalAuthority=None, hasBroaderAuthority=[Uri(value='https://bibliokeia.com/authorities/Topic/bkau-1', label=Label(value='Biblioteconomia', lang='por'), base='bk')], hasBroaderExternalAuthority=[Uri(value='http://id.loc.gov/authorities/subjects/sh85020816', label=Label(value='Cataloging'

In [44]:
request.hasBroaderAuthority

[Uri(value='https://bibliokeia.com/authorities/Topic/bkau-1', label=Label(value='Biblioteconomia', lang='por'), base='bk')]

In [45]:
Narrower = """PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 

INSERT DATA
{ GRAPH <https://bibliokeia.com/authorities/Topic/bkau-1> 
{ <https://bibliokeia.com/authorities/Topic/bkau-1>  madsrdf:hasNarrowerAuthority  <https://bibliokeia.com/authorities/Topic/bkau-1> } }"""

au_update = FusekiUpdate('http://localhost:3030', 'authorities')
r = au_update.run_sparql(Narrower)
r

In [47]:
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
graph = MakeGraphSubject(request, "TESTE")
print(graph)

In [2]:
fuseki_query = FusekiQuery('http://localhost:3030', 'authorities')
token = 'sh85066150'
query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            SELECT ?uri ?label WHERE {{ graph ?g {{
            ?sub bf:identifiedBy ?identifiedBy .
    ?identifiedBy rdf:value "{token}" .
    ?uri madsrdf:authoritativeLabel ?label .   
                }} }} """
responseQuery = fuseki_query.run_sparql(query)
exist = responseQuery.convert()['results']['bindings'][0]

In [4]:
exist = GraphExist("http://id.loc.gov/authorities/subjects/sh85066150")
exist

{'uri': {'type': 'uri',
  'value': 'https://bibliokeia.com/authorities/Topic/bkau-2'},
 'label': {'type': 'literal', 'value': 'Ciência da informação'}}

In [21]:
from api.src.function.loc.CloseExternal import GetCloseExternal

In [19]:
# Type 
def GetType(graph, authority):

  qtype = f"""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
              PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
              PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
              SELECT ?type WHERE {{ 
                  <{authority}> rdf:type ?type .
                  FILTER ( ?type != madsrdf:Authority ) 
                  FILTER ( ?type != skos:Concept )
                  }}"""
  r = graph.query(qtype)
  tipo = r.bindings[0].get('type').split("#")[1]
  return tipo

# hasBroaderAuthority
def GetHasBroader(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasBroaderAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
               print(exist)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(external) > 0:
            obj['hasBroaderExternalAuthority'] = external
    return obj

# hasNarrowerAuthority
def GetHasNarrower(authority, graph, obj):

    ask = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasNarrowerAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(ask)
    if len(response.bindings) > 0:
        external = list()
        internal = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
                uri = {
                   "value": exist['uri']['value'],
                   "base": "bk",
                   "label": {
                       "value": exist['label']['value'],
                       "lang": "pt" } }
                internal.append(uri)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(internal) > 0:
            obj['hasNarrowerAuthority'] = internal
        if len(external) > 0:
            obj['hasNarrowerExternalAuthority'] = external
    return obj

# hasNarrowerExternalAuthority
def GetHasNarrowerExternal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasNarrowerExternalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value')
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
            external.append(uri)
        return external
    else:
        return False

# hasExactExternalAuthority
def GetExactExternal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasExactExternalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           base = url.split("//")[1].split("/")[0]
           uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
           external.append(uri)
        obj['hasExactExternalAuthority'] = external
    return obj

# hasReciprocalAuthority
def GetHasReciprocal(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasReciprocalAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        internal = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
 
           if exist:
                uri = {
                   "value": exist['uri']['value'],
                   "base": "bk",
                   "label": {
                       "value": exist['label']['value'],
                       "lang": "por" } }
                internal.append(uri)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(internal) > 0:
            obj['hasReciprocalAuthority'] = internal
        if len(external) > 0:
            obj['hasReciprocalExternalAuthority'] = external
    return obj

# hasVariant
def GetVariant(authority, graph, obj):

    qVariant = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
                PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
                SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
                <{authority}> madsrdf:hasVariant ?variant .
                ?variant rdf:type ?typeVariant .
                ?variant madsrdf:elementList ?elementList .
                ?elementList rdf:rest* ?node .
                  ?node rdf:first ?e .
                  ?e madsrdf:elementValue ?elementValue .
                ?e rdf:type ?typeElement .
                FILTER ( ?typeVariant != madsrdf:Variant )
                }}"""
    r = graph.query(qVariant)
    if len(r.bindings) > 0:
        translator = MakeTranslate(
            source_language='en',
            target_language='pt',
            timeout=10
        )
        variants = list()
        for i in r.bindings:
            value = translator.translate(i.get('elementValue').value)
            variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': value.capitalize(),
                  'lang': 'pt'
              }
          }]
      }
            variants.append(variant)
        obj['hasVariant'] = variants
    return obj

def GetElementList(authority, graph, obj):

    translator = MakeTranslate(
            source_language='en',
            target_language='pt',
            timeout=10
        )

    qElementList = f"""PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
    r = graph.query(qElementList)
    elementList = list()
    
    for i in r.bindings:
        value = translator.translate(i.get('elementValue').value)
        element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  value.capitalize(),
            "lang": 'pt'
          }
        }
        elementList.append(element)
    obj['elementList'] = elementList
    return obj

In [37]:
def GetInternalUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:has{metadata}Authority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        internal = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
 
           if exist:
                uri = {
                   "value": exist['uri']['value'],
                   "base": "bk",
                   "label": {
                       "value": exist['label']['value'],
                       "lang": "por" } }
                internal.append(uri)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(internal) > 0:
            obj[f'has{metadata}Authority'] = internal
        if len(external) > 0:
            obj[f'has{metadata}ExternalAuthority'] = external
    return obj

def GetExternalUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:has{metadata}Authority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value')
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
            external.append(uri)
        return external
    else:
        return False


def ParserSubject(graph, authority):
      
  prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

  # Type
  tipo = GetType(graph, authority)
  
  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]      
  }
  
  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'
  }
  
  # ElementList
  obj = GetElementList(authority, graph, obj)

  # Note 
  qNote = f"""{prefix}
  SELECT ?note WHERE {{ 
      <{authority}> madsrdf:note ?note .
       }}"""
  r = graph.query(qNote)
  if len(r.bindings) > 0:
     obj['note'] = r.bindings[0].get('note').value

  # hasVariant
  obj = GetVariant(authority, graph, obj)

  # URIS
  # hasReciprocalAuthority
#   obj = GetHasReciprocal(authority, graph, obj)
  obj = GetInternalUri(authority, graph, "Reciprocal", obj)

  # hasBroaderAuthority
  obj = GetInternalUri(authority, graph, "Broader", obj)
  # obj = GetHasBroader(authority, graph, obj)
  
  # Narrower Terms
  obj = GetInternalUri(authority, graph, "Narrower", obj)
  # obj = GetHasNarrower(authority, graph, obj)
  
#   hasNarrower = GetHasNarrowerExternal(authority, graph, obj)
  hasNarrower = GetExternalUri(authority, graph, "NarrowerExternal", obj)
  if hasNarrower:
    obj['hasNarrowerExternalAuthority'] = obj['hasNarrowerExternalAuthority']+hasNarrower
  
  # ExactExternal
#   obj = GetExactExternal(authority, graph, obj)
  exactExternal = GetExternalUri(authority, graph, "ExactExternal", obj)
  if exactExternal:
    obj['hasExactExternalAuthority'] = exactExternal
  
  # CloseExternal
#   obj = GetCloseExternal(authority, graph, obj)
  closeExternal = GetExternalUri(authority, graph, "CloseExternal", obj)
  if closeExternal:
    obj['hasCloseExternalAuthority'] = closeExternal


  response = Subject(**obj)

  return response

response = ParserSubject(graph, authority)


In [40]:
response.hasBroaderAuthority

[Uri(value='https://bibliokeia.com/authorities/Topic/bkau-1', label=Label(value='Biblioteconomia', lang='por'), base='bk')]

In [31]:



obj = GetHasUri(authority, graph, "Broader", {})
obj


{'hasBroaderAuthority': [{'value': 'https://bibliokeia.com/authorities/Topic/bkau-1',
   'base': 'bk',
   'label': {'value': 'Biblioteconomia', 'lang': 'por'}}],
 'hasBroaderExternalAuthority': [{'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85020816'),
   'base': 'loc',
   'label': {'value': 'Cataloging', 'lang': 'en'}}]}

In [27]:
obj

{'hasBroaderExternalAuthority': [{'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85020816'),
   'base': 'loc',
   'label': {'value': 'Cataloging', 'lang': 'en'}}]}

In [23]:
obj

{}

In [42]:

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=response.dict())
response.status_code

TypeError: Object of type date is not JSON serializable